In [1]:
import torch
import numpy as np
import pandas as pd
import matchzoo as mz
print('matchzoo version', mz.__version__)

matchzoo version 1.1.1


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=1))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]
print("`ranking_task` initialized with metrics", ranking_task.metrics)

`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [3]:
print('data loading ...')
train_pack_raw = mz.datasets.wiki_qa.load_data('train', task=ranking_task)
dev_pack_raw = mz.datasets.wiki_qa.load_data('dev', task=ranking_task, filtered=True)
test_pack_raw = mz.datasets.wiki_qa.load_data('test', task=ranking_task, filtered=True)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [4]:
preprocessor = mz.models.MatchPyramid.get_default_preprocessor()

In [5]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8411.71it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5277.38it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 3323443.22it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 10832.68it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5802.73it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 871585.14it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 9960.20it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<0

In [6]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x109a0e1d0>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x132deadd8>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [8]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    batch_size=20,
    resample=True,
    sort=False,
    shuffle=True
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    batch_size=20,
    sort=False,
    shuffle=False
)

In [9]:
padding_callback = mz.models.MatchPyramid.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    stage='dev',
    callback=padding_callback
)

In [10]:
model = mz.models.MatchPyramid()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['kernel_count'] = [16, 32]
model.params['kernel_size'] = [[3, 3], [3, 3]]
model.params['dpool_size'] = [3, 10]
model.params['dropout_rate'] = 0.1

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

MatchPyramid(
  (embedding): Embedding(30058, 300, padding_idx=0)
  (matching): Matching()
  (conv2d): Sequential(
    (0): Sequential(
      (0): ConstantPad2d(padding=(0, 2, 0, 2), value=0)
      (1): Conv2d(1, 16, kernel_size=[3, 3], stride=(1, 1))
      (2): ReLU()
    )
    (1): Sequential(
      (0): ConstantPad2d(padding=(0, 2, 0, 2), value=0)
      (1): Conv2d(16, 32, kernel_size=[3, 3], stride=(1, 1))
      (2): ReLU()
    )
  )
  (dpool_layer): AdaptiveAvgPool2d(output_size=[3, 10])
  (dropout): Dropout(p=0.1, inplace=False)
  (out): Linear(in_features=960, out_features=1, bias=True)
)
Trainable params:  9023161


In [11]:
optimizer = torch.optim.Adam(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=5
)

In [12]:
trainer.run()

[Iter-102 Loss-0.642]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4809 - normalized_discounted_cumulative_gain@5(0.0): 0.546 - mean_average_precision(0.0): 0.5082



[Iter-204 Loss-0.314]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5282 - normalized_discounted_cumulative_gain@5(0.0): 0.5779 - mean_average_precision(0.0): 0.5469



[Iter-306 Loss-0.051]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5248 - normalized_discounted_cumulative_gain@5(0.0): 0.583 - mean_average_precision(0.0): 0.5439



[Iter-408 Loss-0.016]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5113 - normalized_discounted_cumulative_gain@5(0.0): 0.5663 - mean_average_precision(0.0): 0.5263



[Iter-510 Loss-0.019]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4994 - normalized_discounted_cumulative_gain@5(0.0): 0.5559 - mean_average_precision(0.0): 0.5164

Cost time: 111.33789110183716s
